In [28]:
import os
from dotenv import load_dotenv

In [29]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [30]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="llama-3.1-8b-instant", model_provider="groq")

In [31]:
name = model.invoke("I want to open a restaurant for Indian food. Suggest me one fancy name for this (in 3 words maximum).")
print(name)

content='"Tandoori Nights"' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 60, 'total_tokens': 66, 'completion_time': 0.011615561, 'prompt_time': 0.00437065, 'queue_time': 0.266416607, 'total_time': 0.015986211}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--c34e0295-c541-436f-92e3-1a0c8d77dc32-0' usage_metadata={'input_tokens': 60, 'output_tokens': 6, 'total_tokens': 66}


In [32]:
from langchain_core.prompts import PromptTemplate

In [33]:
system_template = "Suggest one fancy and unique restaurant name (maximum 3 words) for a restaurant specializing in {cuisine} cuisine. Respond with only the name, nothing else."

prompt_template_name = PromptTemplate.from_template(system_template)

In [34]:
prompt_template_name.invoke({"cuisine": "Italian"})

StringPromptValue(text='Suggest one fancy and unique restaurant name (maximum 3 words) for a restaurant specializing in Italian cuisine. Respond with only the name, nothing else.')

In [35]:
from langchain_core.output_parsers import StrOutputParser

In [36]:
chain_name = prompt_template_name | model | StrOutputParser()
chain_name.invoke({"cuisine": "Indian"})

'Midnight Maha Bazaar'

**Generate menu items from the restaurant name.**

In [37]:
model.invoke("Suggest menu items for 'Mumbai Spice Bazaar'. Respond with only a comma-separated list of items, no explanations or extra text.")

AIMessage(content='Samosas, Chicken Tikka, Biryani, Tandoori Chicken, Palak Paneer, Baingan Bharta, Garlic Naan, Gobhi Manchurian, Mysore Dosa, Raita, Hyderabadi Haleem, Vegetable Korma, Malabar Lamb Curry.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 63, 'total_tokens': 131, 'completion_time': 0.102040435, 'prompt_time': 0.003411263, 'queue_time': 0.216736579, 'total_time': 0.105451698}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a6b69223-f962-4a88-b7bc-e21f62b4f21f-0', usage_metadata={'input_tokens': 63, 'output_tokens': 68, 'total_tokens': 131})

In [38]:
menu_items_template = "Suggest menu items for {restaurant_name}. Respond with only a comma-separated list of 10 items, no explanations or extra text."
prompt_template_menu = PromptTemplate.from_template(menu_items_template)

In [39]:
prompt_template_menu.invoke({"restaurant_name": "Mumbai Spice Bazaar"})

StringPromptValue(text='Suggest menu items for Mumbai Spice Bazaar. Respond with only a comma-separated list of 10 items, no explanations or extra text.')

In [ ]:
chain_menu = prompt_template_menu | model | StrOutputParser()

chain_restaurant_menu = {"restaurant_name" : chain_name} | chain_menu

Spiced Lamb Koftas, Chicken Tikka Masala, Saffron Risotto, Cardamom Crème Brûlée, Garam Masala Wings, Kashmiri Chicken, Moroccan Chickpea Stew, Coconut Shrimp Curry, Ras el Hanout Falafel, Star Anise Beef Tacos


In [48]:
response = chain_restaurant_menu.invoke({"cuisine": "Indian"})

print(response)

Butter Chicken, Chicken Tikka Masala, Palak Paneer, Saag Aloo, Vegetable Biryani, Tandoori Chicken, Malai Kofta, Baingan Bharta, Chana Masala, Hyderabadi Dum Biryani


In [77]:
from langchain_core.runnables import RunnablePassthrough

In [78]:
#  For both the outputs
full_chain = {"restaurant_name" : chain_name} | RunnablePassthrough.assign(menu_items = chain_menu)

result = full_chain.invoke({"cuisine": "Indian"})
print(result)

{'restaurant_name': '"Maharaja\'s Spice Court"', 'menu_items': 'Chicken Tikka Masala, Palak Paneer, Naan Bread, Biryani, Butter Chicken, Samosas, Raita, Gulab Jamun, Tandoori Chicken, Ras Malai'}


In [79]:
description_template = "Write a short and simple 2-sentence description for {restaurant_name} that serves: {menu_items}"

prompt_template_description = PromptTemplate.from_template(description_template)

In [80]:
chain_description = prompt_template_description | model | StrOutputParser()

In [81]:
combined_chain = {"restaurant_name" : chain_name} | RunnablePassthrough.assign(menu_items = chain_menu) | RunnablePassthrough.assign(description = chain_description)

In [82]:
combined_chain.invoke({
    "cuisine" : "Indian"
})

{'restaurant_name': 'Desert Spice Bazaar',
 'menu_items': 'Saffron-infused Chicken Biryani, Cardamom Crème Brûlée, Rosewater Panna Cotta, Pistachio Kulfi, Coconut Ladoo, Ginger and Honey Cake, Lemon and Rose Petal Sorbet, Cinnamon Swirl Coffee Cake, Orange Blossom Pudding, Chai Spice Crème Caramel',
 'description': 'Desert Spice Bazaar is a charming eatery offering a rich and exotic array of Middle Eastern-inspired desserts and international fusion sweets. From traditional Saffron-infused Chicken Biryani to innovative desserts like Cardamom Crème Brûlée and Rosewater Panna Cotta, every treat is infused with unique spices and flavors.'}